# Train

In [ ]:
# Mount your Google Drive if needed

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Note: Some libraries or dependencies used in this code may become deprecated over time.
# If you encounter deprecated libraries, replace them with suitable alternatives and
# update the corresponding sections of the code that rely on the deprecated classes or methods accordingly.

!pip install transformers[torch]==4.31.0
!pip install datasets

import os
import json
import gzip
import pandas as pd
import collections
import nltk
from google.colab import files

from urllib.request import urlopen

import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import pickle
from IPython.display import clear_output
import torch
from transformers import  AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, AutoConfig
from datasets import load_metric

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices: {}".format(DEVICE))
print(torch.cuda.get_device_name())


# Comment out any models in the list below to exclude them from training.
all_models = [
    "microsoft/deberta-base",
    "microsoft/deberta-large",
    "xlnet/xlnet-large-cased",
    "albert/albert-large-v2",
    "FacebookAI/xlm-roberta-large",
    "FacebookAI/xlm-roberta-base",
    "distilbert/distilroberta-base",
    "albert/albert-base-v2",
    "FacebookAI/roberta-large"
        ]


def set_seed(SEED):
    """ Set random seed to all """
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True


seeds = [
    323, 6932, 200, 5, 94, 1759, 1694, 9741, 791, 999
    ]  # randomly selected seeds for the study


class AmazonDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


def prepare_auxilary_datasets(dataset):

    X_test =  dataset["sentence"]
    y_test =  dataset["label"]   if "label" in dataset.columns.values else   dataset["helpful"]
    # Encoding
    test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=40)
    test_dataset = AmazonDataset(test_encodings, y_test.tolist())
    return test_dataset

def compute_metrics(eval_pred):

    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("accuracy")
    metric4 = load_metric("f1")
    metric5 = load_metric("roc_auc")

    logits  = eval_pred.predictions
    labels   = eval_pred.label_ids
    predictions = np.argmax(logits, axis=-1)

    precision = metric1.compute(predictions=predictions, references=labels)["precision"]
    recall = metric2.compute(predictions=predictions, references=labels)["recall"]
    accuracy = metric3.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = metric4.compute(predictions=predictions, references=labels)["f1"]
    auc = metric5.compute(prediction_scores=logits[:, 1], references=labels)["roc_auc"]
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'AUC': auc
    }


training_args = TrainingArguments(
    output_dir="test-amazon",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=128,
    num_train_epochs =5,
    weight_decay=0.01,
    load_best_model_at_end=False,
    logging_strategy = "steps",
    logging_steps = 100,
    save_strategy= "no",
)





Devices: cuda


In [ ]:

# Ensure you have access to the datasets and update the directory paths as needed.
data_set_1 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/8000_sentences_with_need_IDs.csv')
data_set_2 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/Electronics.csv')
data_set_3 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/baby.csv')
data_set_4 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/Pet_supplies.csv')
data_set_5 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/Sport_outdoors.csv')

# Ensure the 'sentence' column exists and is of type 'str'.
# Rename or update the column if needed.
data_set_1['sentence'] = data_set_1['sentence'].astype(str)
data_set_2['sentence'] = data_set_2['sentence'].astype(str)
data_set_3['sentence'] = data_set_3['sentence'].astype(str)
data_set_4['sentence'] = data_set_4['sentence'].astype(str)
data_set_5['sentence'] = data_set_5['sentence'].astype(str)


# Custom settings for learning rates and epochs have been applied in this script to reduce the
# computational overhead, as training with the full set of hyperparameters is exhaustive.
# If you wish to reproduce the full results as presented in the paper, please refer to the
# detailed hyperparameter settings provided in the paper and modify the following nested
# for loop to include those settings.


# Sample run
for model_name in all_models:
    if model_name ==  "FacebookAI/xlm-roberta-large":
       ml_name = "xlm-r-large"
       training_args.learning_rate = 8e-6
       training_args.num_train_epochs = 7
    elif model_name ==  "FacebookAI/xlm-roberta-base":
       ml_name = "xlm-r-base"
       training_args.learning_rate = 2e-5
       training_args.num_train_epochs = 5
    elif model_name == "distilbert/distilroberta-base":
       ml_name = "disRobert-B"
       training_args.learning_rate = 2e-5
       training_args.num_train_epochs = 5
    elif model_name == "albert/albert-base-v2":
       ml_name = "albert-base"
       training_args.learning_rate = 1e-5

    elif model_name == "FacebookAI/roberta-large":
       ml_name = "roberta-large"
       training_args.learning_rate = 1e-5


    tokenizer = AutoTokenizer.from_pretrained(model_name)
    config = AutoConfig.from_pretrained(
        model_name, num_labels=2
    )
    all_datasets_seed_fold_results = {}
    results = []
    for seed in seeds:
        training_args.seed = seed
        set_seed(seed)
        all_datasets_seed_fold_results.update({seed:{}})
        fold_counter = 1
        kf = KFold(n_splits=5, random_state= seed, shuffle=True) # Define the split - into 5 folds
        for train_index, test_index in kf.split(data_set_1):
            X_train, X_test = data_set_1.loc[train_index, "sentence"], data_set_1.loc[test_index, "sentence"]
            y_train, y_test = data_set_1.loc[train_index, "label"], data_set_1.loc[test_index, "label"]

            # Encoding
            train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=40)
            test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=40)
            train_dataset = AmazonDataset(train_encodings, y_train.tolist())
            test_dataset = AmazonDataset(test_encodings, y_test.tolist())

            print(
                f'Model seed is: {training_args.seed}',
                f'Model LR is: {training_args.learning_rate}'
                  )
            model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                       config=config
                                                                       )
            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=train_dataset,
                #eval_dataset=val_dataset,             # evaluation dataset must be off when doing simple K-fold cross validation
                # callbacks= [EarlyStoppingCallback(early_stopping_patience= 3,early_stopping_threshold=0.1)],  # only needed for full training
                compute_metrics=compute_metrics
            )
            trainoutput = trainer.train()
            tr_loss = trainoutput.training_loss
            pred = trainer.predict(test_dataset)
            # NOTE: "skin-care" is a typo throughout the entire code.
            # It refers to the Oral-Care dataset mentioned in the paper.
            results.append(["skin-care",seed, fold_counter, pred.metrics['test_accuracy'], pred.metrics['test_f1'], pred.metrics['test_precision'], pred.metrics['test_recall'], pred.metrics['test_AUC']])
            all_datasets_seed_fold_results[seed].update( {fold_counter : {"skin-care": [train_index, test_index, pred, tr_loss],
                                                                          "electronics": trainer.predict(prepare_auxilary_datasets(data_set_2)),
                                                                          "baby": trainer.predict(prepare_auxilary_datasets(data_set_3)),
                                                                          "pet-supplies": trainer.predict(prepare_auxilary_datasets(data_set_4)),
                                                                          "Sport-outdoors": trainer.predict(prepare_auxilary_datasets(data_set_5)),
                                                                          }
                                                          })
            print(f'seed is {seed} in fold {fold_counter}')
            fold_counter +=1
    df_result = pd.DataFrame(results, columns = ['category', 'seed', 'fold', 'accuracy','f1-score', 'precision', 'recall', 'AUC'])
    display(df_result)
    display(df_result.groupby(df_result.category).mean())

    tables = {
        "skin-care": results,"electronics": [],"baby": [],
        "pet-supplies": [],"Sport-outdoors": []
        }


    for seed, values in all_datasets_seed_fold_results.items():
        for fold, val in values.items():
            for dataset, v in val.items():
                if dataset == "skin-care":
                  tables["skin-care"] =  df_result
                else:
                   tables[dataset].append([dataset, seed, fold, v.metrics['test_accuracy'],
                                           v.metrics['test_f1'], v.metrics['test_precision'],
                                           v.metrics['test_recall'], v.metrics['test_AUC']])

    df_total_list = [pd.DataFrame(t, columns = ['category', 'seed', 'fold', 'accuracy','f1-score', 'precision', 'recall', 'AUC']) for k, t in tables.items()]
    df = pd.concat(df_total_list).drop_duplicates().reset_index(drop=True)
    display(df.groupby(df.category).mean())

    lr_ = training_args.learning_rate
    epc = training_args.num_train_epochs


    # Specify the path ( here Google Drive has been utilized)
    ############### google drive  ########
    path_df = f'/content/drive/MyDrive/paper1/results_exp1/df_{ml_name}_{lr_}_16_{epc}.pkl'
    # Save the DataFrame as a pickle file
    df_result.to_pickle(path_df)

    # Specify the path ( here Google Drive has been utilized)
    file_path = f'/content/drive/MyDrive/paper1/results_exp1/all_datasets_seed_fold_results_{ml_name}_{lr_}_16_{epc}.pkl'
    # Serialize and save the dictionary
    with open(file_path, 'wb') as handle:
        pickle.dump(all_datasets_seed_fold_results, handle, protocol=pickle.HIGHEST_PROTOCOL)





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model seed is: 323 Model LR is: 1e-05


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,0.646400
200,0.553600
300,0.527000
400,0.491700
500,0.419600
600,0.400600
700,0.393800
800,0.374000
900,0.279000
1000,0.279500


<ipython-input-3-8c677c5ec465>:54: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric1 = load_metric("precision")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/precision/precision.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for recall contains custom code which must be executed to correctly load the metric. You can inspect the

...

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/precision/precision.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for recall contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/recall/recall.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric fro

seed is 323 in fold 5


,category,seed,fold,accuracy,f1-score,precision,recall,AUC
0,skin-care,323,1,0.800000,0.804640,0.820672,0.789222,0.882942
1,skin-care,323,2,0.770625,0.774431,0.759952,0.789474,0.847785
2,skin-care,323,3,0.801250,0.815331,0.783482,0.849879,0.871907
3,skin-care,323,4,0.804375,0.815991,0.813599,0.818396,0.880006
4,skin-care,323,5,0.794375,0.809275,0.820212,0.798627,0.872047


,seed,fold,accuracy,f1-score,precision,recall,AUC
category,,,,,,,
skin-care,323.0,3.0,0.794125,0.803934,0.799583,0.809119,0.870937


,seed,fold,accuracy,f1-score,precision,recall,AUC
category,,,,,,,
Sport-outdoors,323.0,3.0,0.766400,0.682750,0.684320,0.681301,0.801358
baby,323.0,3.0,0.724300,0.712720,0.645071,0.799299,0.797831
combined-categories,323.0,3.0,0.559500,0.526424,0.886252,0.375095,0.717050
electronics,323.0,3.0,0.750400,0.654693,0.717258,0.603308,0.799105
pet-supplies,323.0,3.0,0.786400,0.700898,0.633857,0.784664,0.846716
skin-care,323.0,3.0,0.794125,0.803934,0.799583,0.809119,0.870937


In [ ]:
df_result

,category,seed,fold,accuracy,f1-score,precision,recall,AUC
0,skin-care,323,1,0.800000,0.804640,0.820672,0.789222,0.882942
1,skin-care,323,2,0.770625,0.774431,0.759952,0.789474,0.847785
2,skin-care,323,3,0.801250,0.815331,0.783482,0.849879,0.871907
3,skin-care,323,4,0.804375,0.815991,0.813599,0.818396,0.880006
4,skin-care,323,5,0.794375,0.809275,0.820212,0.798627,0.872047


# Inference

In [ ]:
# Load the results from the specified directories.

df_train_test = pd.read_csv('/content/sample_data/8000_sentences_with_need_IDs.csv')
# example
df_result_allenai_rb = pd.read_pickle('/content/sample_data/1-df_allenai_roberta_base')
frequencies = pd.read_pickle('/content/sample_data/dictionary frequency of occurence of needs')
all_needs = pd.read_pickle('/content/sample_data/all_needs_ids')

all_datasets_seed_fold_results_allenai_roberta_base =  pd.read_pickle('/content/sample_data/1-all_datasets_seed_fold_results_allenai_roberta_base')
display(df_result_allenai_rb)
display(df_result_allenai_rb.groupby(df_result_allenai_rb.category).mean())
results = df_result_allenai_rb[df_result_allenai_rb["category"] == 'skin-care']
tables = {"skin-care": results,"electronics": [],"baby": [],
          "pet-supplies": [],"Sport-outdoors": []}


for seed, values in all_datasets_seed_fold_results_allenai_roberta_base.items():
    for fold, val in values.items():
        for dataset, v in val.items():
            if dataset == "skin-care":
              tables["skin-care"] =  df_result_allenai_rb
            else:
               tables[dataset].append([dataset, seed, fold, v.metrics['test_accuracy'],
                                       v.metrics['test_f1'], v.metrics['test_precision'],
                                       v.metrics['test_recall'], v.metrics['test_AUC']])

df_total_list = [pd.DataFrame(t, columns = ['category', 'seed', 'fold', 'accuracy','f1-score', 'precision', 'recall', 'AUC']) for k, t in tables.items()]
df = pd.concat(df_total_list).drop_duplicates().reset_index(drop=True)
df.groupby(df.category).mean()